# 01 - Binomial Pricing and Replication

Implement CRR binomial pricing, inspect replication quantities, and verify convergence to Black-Scholes.

In [5]:
%cd /content
!git clone https://github.com/basarr/interactive_portfolio_optimization.git
%cd /content/interactive_portfolio_optimization
!ls


/content
Cloning into 'interactive_portfolio_optimization'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (49/49), done.
Receiving objects: 100% (55/55), 36.73 KiB | 596.00 KiB/s, done.
remote: Total 55 (delta 10), reused 44 (delta 4), pack-reused 0 (from 0)
Resolving deltas: 100% (10/10), done.
/content/interactive_portfolio_optimization
data  notebooks  pyproject.toml  README.md  results  src  tests


In [6]:
import sys
ROOT = Path("/content/interactive_portfolio_optimization")
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
print("ROOT:", ROOT)


ROOT: /content/interactive_portfolio_optimization


In [7]:
from pathlib import Path
import sys

candidates = [
    Path.cwd(),
    Path.cwd().parent,
    Path("/content/Interactive_Portfolio_Optimization"),
    Path("/content/derivatives-risk-lab"),
]

ROOT = next((p for p in candidates if (p / "src").exists()), None)
if ROOT is None:
    raise FileNotFoundError("Could not find project root containing /src")

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("ROOT =", ROOT)
print("src exists:", (ROOT / "src").exists())


ROOT = /content/interactive_portfolio_optimization
src exists: True


In [8]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.config import config_dict
from src.binomial import replication_tree, convergence_table_to_bs
from src.plotting import plot_binomial_convergence

cfg = config_dict(fast_mode=True)


In [9]:
small = replication_tree(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'], N=5, option_type='call'
)
print('Root tree price:', small['price'])
print('Root replication price:', small['root_replication_price'])
print('Root replication gap:', small['root_replication_gap'])

node_rows = []
for step, vals in enumerate(small['option_values']):
    for node, val in enumerate(vals):
        node_rows.append({'step': step, 'node': node, 'option_value': float(val)})
node_df = pd.DataFrame(node_rows)

fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(node_df['step'], node_df['option_value'], alpha=0.8)
ax.set_title('Option Values Across Nodes (N=5)')
ax.set_xlabel('Step')
ax.set_ylabel('Option Value')
fig.tight_layout()
fig.savefig(ROOT / 'results' / 'figures' / 'binomial_nodes_n5.png', dpi=150)
plt.close(fig)

delta_rows = []
for step, vals in enumerate(small['deltas']):
    for node, val in enumerate(vals):
        delta_rows.append({'step': step, 'node': node, 'delta': float(val)})
delta_df = pd.DataFrame(delta_rows)

fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(delta_df['step'], delta_df['delta'], alpha=0.8)
ax.set_title('Deltas Across Nodes (N=5)')
ax.set_xlabel('Step')
ax.set_ylabel('Delta')
fig.tight_layout()
fig.savefig(ROOT / 'results' / 'figures' / 'binomial_deltas_n5.png', dpi=150)
plt.close(fig)


Root tree price: 9.308750330429364
Root replication price: 9.308750330429362
Root replication gap: -1.7763568394002505e-15


In [10]:
N_values = [10, 25, 50, 100, 200, 400]
conv = convergence_table_to_bs(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'], option_type='call', N_values=N_values
)
conv.to_csv(ROOT / 'results' / 'tables' / 'binomial_convergence.csv', index=False)
plot_binomial_convergence(conv, ROOT / 'results' / 'figures' / 'binomial_convergence.png')
conv


,N,binomial_price,bs_price,abs_error,signed_error
0,10,8.720683,8.916037,0.195354,-0.195354
1,25,8.993558,8.916037,0.077521,0.077521
2,50,8.876513,8.916037,0.039524,-0.039524
3,100,8.896250,8.916037,0.019788,-0.019788
4,200,8.906137,8.916037,0.009900,-0.009900
5,400,8.911086,8.916037,0.004952,-0.004952
